In [8]:
import pandas as pd

# Load the Excel file
excel_data = pd.read_excel("Q1_未来30天各分拣中心预测货量数据.xlsx")

# Load the two CSV files with GBK encoding
csv_data_3 = pd.read_csv("附件/附件3.csv", encoding="gbk")
csv_data_4 = pd.read_csv("附件/附件4.csv", encoding="gbk")


思路：

1、根据如附件3中的货量分布来估算各运输线路的货量占比比例；

2、对比附件3与附件4，分析未来30天的新增和停止的运输线路；

3、读取未来30天各分拣中心每天的预测货量数据，货量比的调整：
- 对于新增的运输线路，线路因增加而需要额外的货量，可以根据步骤1中的比例，相应增加对应比例的货量，新增的运输线路货量为0，可以设置始发分拣中心到其他到达分拣中心 的货量均值作为货量比例
- 对于停止的运输线路，可以根据步骤1中的比例，相应减少对应比例的货量。

In [15]:
# 步骤1: 计算附件3中各运输线路的货量占比
csv_data_3['货量比例'] = csv_data_3['货量'] / csv_data_3['货量'].sum()
csv_data_3

,始发分拣中心,到达分拣中心,货量,货量比例
0,SC22,SC8,296,0.015598
1,SC18,SC8,213,0.011224
2,SC7,SC8,240,0.012647
3,SC15,SC8,172,0.009064
4,SC52,SC8,72,0.003794
...,...,...,...,...
129,SC9,SC53,184,0.009696
130,SC31,SC53,220,0.011593
131,SC53,SC9,199,0.010486
132,SC34,SC9,214,0.011277


In [16]:
# 步骤2: 对比附件3与附件4，分析新增和停止的运输线路
routes_attachment_3 = set(zip(csv_data_3['始发分拣中心'], csv_data_3['到达分拣中心']))
routes_attachment_4 = set(zip(csv_data_4['始发分拣中心'], csv_data_4['到达分拣中心']))
new_routes = routes_attachment_4 - routes_attachment_3
discontinued_routes = routes_attachment_3 - routes_attachment_4

print("新增的运输线路:")
new_routes


新增的运输线路:


{('SC31', 'SC9'), ('SC5', 'SC4')}

In [17]:
print("停止的运输线路:")
discontinued_routes

停止的运输线路:


{('SC1', 'SC25'),
 ('SC18', 'SC51'),
 ('SC19', 'SC15'),
 ('SC2', 'SC19'),
 ('SC24', 'SC5'),
 ('SC28', 'SC4'),
 ('SC36', 'SC47'),
 ('SC36', 'SC8'),
 ('SC39', 'SC60'),
 ('SC4', 'SC15'),
 ('SC51', 'SC15'),
 ('SC54', 'SC25'),
 ('SC55', 'SC7'),
 ('SC61', 'SC10')}

In [18]:
# 准备数据集
new_routes_df = pd.DataFrame(list(new_routes), columns=['始发分拣中心', '到达分拣中心'])
discontinued_routes_df = pd.DataFrame(list(discontinued_routes), columns=['始发分拣中心', '到达分拣中心'])
discontinued_routes_df

,始发分拣中心,到达分拣中心
0,SC4,SC15
1,SC18,SC51
2,SC28,SC4
3,SC39,SC60
4,SC54,SC25
5,SC19,SC15
6,SC51,SC15
7,SC1,SC25
8,SC55,SC7
9,SC36,SC8


In [19]:
# 步骤3: 为新增路线估算货量，使用始发分拣中心的平均货量和比例
average_volume_proportion = csv_data_3.groupby('始发分拣中心').agg({'货量': 'mean', '货量比例': 'mean'}).reset_index()
new_routes_estimated = new_routes_df.merge(average_volume_proportion, how='left', on='始发分拣中心')
new_routes_estimated.rename(columns={'货量': '估算货量', '货量比例': '估算货量比例'}, inplace=True)
new_routes_estimated.fillna({'估算货量': 0, '估算货量比例': 0}, inplace=True)
new_routes_estimated#新增的运输线路及其估算货量和货量比例:

,始发分拣中心,到达分拣中心,估算货量,估算货量比例
0,SC31,SC9,220.0,0.011593
1,SC5,SC4,32.0,0.001686


In [20]:

print("\n停止的运输线路:")
discontinued_routes_df


停止的运输线路:


,始发分拣中心,到达分拣中心
0,SC4,SC15
1,SC18,SC51
2,SC28,SC4
3,SC39,SC60
4,SC54,SC25
5,SC19,SC15
6,SC51,SC15
7,SC1,SC25
8,SC55,SC7
9,SC36,SC8


In [21]:
# 应用货量调整到未来30天的预测数据
# 计算新增和停止线路的货量调整
new_volume_adjustment = new_routes_estimated['估算货量比例'].sum()
discontinued_volume_adjustment = discontinued_routes_df.merge(csv_data_3, how='left', on=['始发分拣中心', '到达分拣中心'])['货量比例'].sum()
# 调整货量
adjusted_forecast = excel_data['货量'] * (1 + new_volume_adjustment - discontinued_volume_adjustment)
excel_data['调整后货量'] = adjusted_forecast

In [22]:
print("\n调整后的未来30天预测货量:")
excel_data[['日期', '分拣中心', '货量', '调整后货量']]


调整后的未来30天预测货量:


,日期,分拣中心,货量,调整后货量
0,1,SC48,812.352202,736.669076
1,2,SC48,806.396235,731.267999
2,3,SC48,800.440269,725.866922
3,4,SC48,794.484302,720.465845
4,5,SC48,788.528335,715.064769
...,...,...,...,...
1705,26,SC25,39216.132512,35562.545418
1706,27,SC25,38887.909709,35264.901628
1707,28,SC25,38561.670126,34969.056289
1708,29,SC25,38238.405372,34675.908628


In [23]:
# 保存调整后的数据到新的Excel文件
excel_data.to_excel("Q2_调整后的未来30天货量预测.xlsx", index=False)